Now we look at making stacked histograms of the signals, and then a plot of the spectrum with signal, data and a fitted background function.

This example uses ATLAS open data from 2015+2016. We are looking at a simple version of the Higgs->GammaGamma discovery anlaysis.
(https://arxiv.org/pdf/1408.7084)

Things to try:
- where there are #TODOs in the code
- Look for chunks of code that are repeated between these 2 functions - how could we make the overal script shorter using functions/classes?
- if this was an actually python script, not a jupyeter notebook, how might you change the way we are setting up our hard-coded settings?
- we already gave an x-axis label in the histogram production but what if we want to override that now .eg. to change mGamGam to m_{GamGam} or M(GamGam)?
- do you understand what's happening to the overflow and underflow bins? 

In [ ]:
import ROOT as r
from ROOT import TFile, TCanvas, TLegend, TLatex, THStack, TPad, TF1, TGraphAsymmErrors

import sys
sys.path.insert(0, '../utils/')

from utils import createDirs

import os

# Can use a macro to set a lot of style settings you want, experiments might have these to ensure nice uniformity. e.g. here we use the ATLAS one, but you could have your own to get a reliable setup you're happy with.
r.gROOT.LoadMacro("../utils/AtlasStyle.C")
r.SetAtlasStyle()
r.gROOT.SetBatch(1)
r.gStyle.SetOptStat(0)
r.gStyle.SetPalette(112)
r.gStyle.SetTitleYOffset(1.1)

luminosity_ifb = 10

# Define out input and output paths, make sure the output path exists.
hist_path = "../histograms/GamGam_root/"
plot_path = "../plots/GamGam_root/"
createDirs(plot_path)



Applying ATLAS style settings...

/plots
..//plots
/plots/GamGam_root
..//plots/GamGam_root
/plots/GamGam_root/
..//plots/GamGam_root/


In [ ]:
def plotStack(hists_in, xrange, plotdir, variable, rebin=1):
    """
    plot a stack of filled histograms

    Args:
        hists_in (dict(str,TH1)): str labels of input hists and the hists themselves.
        xrange (list(double)): min and max x-range.
        plotdir (str): directory to save plot to.
        variable (str): name of variable to plot.
        rebin (int): to rebin newbinwidth = rebin * oldbinwidth

    """
    if len(xrange)<2: raise IndexError
    if int(rebin)!=rebin: raise Exception("need rebin to be an integer")

    # define some config
    legenddict = {"ggfHiggs": "ggf H#gamma#gamma", 
                  "VBFHiggs": "VBF H#gamma#gamma"}
    colourdict = {"ggfHiggs": r.kSpring-5,
                  "VBFHiggs": r.kMagenta-6}

    # initialise our canvas
    canvas = TCanvas(variable, variable, 200, 10, 700, 750)
    canvas.cd(1) # point current root tdirectory and Draw()s to it
    canvas.SetTopMargin(0.03)
    canvas.SetRightMargin(0.05)
    canvas.SetLeftMargin(0.12)
    canvas.SetFillColor(r.kWhite)
    canvas.SetFillStyle(0)
    canvas.SetLineWidth(1)
    canvas.SetLogy(1)

    # intiialise our legend
    legend = TLegend(0.7, 0.8, 0.95, 0.94)
    legend.SetTextFont(42)
    legend.SetTextSize(0.036)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
  
    # initalise our Stack
    stack = THStack("signal_stack", ";;Events / bin")

    # read in and config our histograms
    hist = {}
    for label, hist_in in hists_in.items():
        hist[label] = hist_in.Clone(label)
        hist[label].Sumw2()
        if rebin!=1: hist[label].Rebin(rebin)
        hist[label].GetXaxis().SetRangeUser(xrange[0], xrange[1])
        hist[label].SetFillColor(colourdict[label])
        stack.Add(hist[label])
        legend.AddEntry(hist[label], legenddict[label], "f") # filled

    # Get the stack total
    stack_total = stack.GetStack().Last().Clone("total")
    stack_total.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    stack_total.SetFillStyle(0) # 0 = hollow
     # Extract error on SM total
    total_error = TGraphAsymmErrors(stack_total)
    total_error.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    total_error.SetFillStyle(3005)
    total_error.SetMarkerSize(0)
    total_error.SetFillColor(r.kBlack)
    total_error.SetLineColor(r.kBlack)
    legend.AddEntry(total_error,"Total","flp") # fill, line, marker

    # find the y range and set nice axis limits around it
    stack_total.GetYaxis().SetRangeUser(max(stack_total.GetMinimum(),0.001)*0.2,stack_total.GetMaximum()*10)
    
    # actually draw the histograms in the canvas (https://root.cern/doc/v628/classTHistPainter.html)
    stack_total.Draw("hist") # plot this first to set the y range
    stack.Draw("histf same") # filled hist
    total_error.Draw("E2")
    # draw the legend
    legend.Draw("same")

    # add some text
    l=TLatex()
    l.SetNDC()
    l.SetTextFont(42)
    l.SetTextColor(r.kBlack)
    l.SetTextSize(0.036)
    lumi_string = '10 fb^{#minus1} #sqrt{s}=13 TeV'
    l.DrawLatex(0.2, 0.9, lumi_string)
 
    canvas.RedrawAxis()
    exts = [".pdf", ".png"]
    for ext in exts: canvas.SaveAs(plotdir + "Stack_" + variable + ext)

In [3]:
# config our samples and variables
samples_to_stack = ["VBFHiggs", "ggfHiggs"]
vars_to_plot = ["diphoton_mass"]

# Note we already nicely defined our x-axis label and binning in the previous script, but you could override/refine that here.
var_config = {
    "diphoton_mass":{
        "xrange": [0., 500.],
        "rebin": 5
    }
}

# TODO can you alter the function to allow us to use variable size bins?
# rebin docs here: https://root.cern.ch/doc/master/classTH1.html#a9eef6f499230b88582648892e5e4e2ce


# retrieve the histogram files
files = {}
for sample in samples_to_stack:
    files[sample] = TFile(hist_path + sample + ".root", "READ")
    print(files[sample])
for var in vars_to_plot:
    # produce dict of our histograms per sample 
    hists_in = {}
    for sample in samples_to_stack:
        hists_in[sample] = files[sample].Get(var)
        print(hists_in[sample])
    # produce the stack plots
    plotStack(hists_in, var_config[var]["xrange"], plot_path, var, rebin=var_config[var]["rebin"])

Name: ../histograms/GamGam_root/VBFHiggs.root Title: 
Name: ../histograms/GamGam_root/ggfHiggs.root Title: 
Name: diphoton_mass Title: diphoton_mass NbinsX: 1000
Name: diphoton_mass Title: diphoton_mass NbinsX: 1000


Warning in <TH1F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: pdf file ../plots/GamGam_root/Stack_diphoton_mass.pdf has been created
Info in <TCanvas::Print>: png file ../plots/GamGam_root/Stack_diphoton_mass.png has been created


In [ ]:
def plotSigBgData(sig_hist_in, data_hist_in, xrange, plotdir, variable, blinded, range_to_blind=[110., 140.]):
    """
    plot a signal and data, and fit/plot a parametrised background estimate

    Args:
        sig_hists (TH1): signal histogram
        data_hist (TH1): data histogram
        xrange (list(double)): min and max x-range.
        plotdir (str): directory to save plot to
        variable (str): name of variable to plot
        blinded (bool): should we blind the data around the signal?
    """
    if len(xrange)<2: raise IndexError

    # initialise our canvas
    canvas = TCanvas(variable, variable, 200, 10, 700, 750)
    canvas.cd(1) # point current root tdirectory and Draw()s to it
    canvas.SetTopMargin(0.03)
    canvas.SetRightMargin(0.05)
    canvas.SetLeftMargin(0.12)
    canvas.SetFillColor(r.kWhite)
    canvas.SetFillStyle(0)
    canvas.SetLineWidth(1)

    # we want 2 pads, for the distribution and ratios.
    pad1 = TPad("upper_pad","upper_pad",0.0,0.3,1.0,1.0,21)
    pad2 = TPad("lower_pad","lower_pad",0.0,0.0,1.0,0.295,22)
    for pad in [pad1, pad2]:
        pad.Draw()
        pad.SetRightMargin(0.03)
        pad.SetFillColor(r.kWhite)
        pad.SetLeftMargin(0.14)
    pad1.SetTopMargin(0.02)
    pad1.SetBottomMargin(0.00)
    pad2.SetTopMargin(0.00)
    pad2.SetBottomMargin(0.3)
    pad1.cd()

    # intiialise our legend
    legend = TLegend(0.6, 0.8, 0.9, 0.94)
    legend.SetTextFont(42)
    legend.SetTextSize(0.036)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)

    # sort out the data histograms
    data_hist = data_hist_in.Clone("data")
    data_hist.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    data_hist.Sumw2()
    data_hist.SetMarkerColor(r.kBlack)
    data_hist.GetYaxis().SetTitleOffset(1.5)
    data_hist.SetMarkerStyle(20) # https://root.cern.ch/doc/master/classTAttMarker.html 
    if blinded and variable=="diphoton_mass":
        minbin = data_hist.FindBin(range_to_blind[0])
        maxbin = data_hist.FindBin(range_to_blind[1])
        for bin in range(minbin, maxbin+1): data_hist.SetBinContent(bin, 0.)
    legend.AddEntry(data_hist, "data", "ep") # lep = line, error, points

    # sort out the signal histogram
    sig_hist = sig_hist_in.Clone("signal")
    sig_hist.Sumw2()
    sig_hist.GetXaxis().SetRangeUser(xrange[0], xrange[1])
    sig_hist.SetLineColor(r.kRed)
    sig_hist.SetLineStyle(1)
    sig_hist.SetLineWidth(3)
    sig_hist.SetMarkerSize(0)
    legend.AddEntry(sig_hist, "ggf + VBF SM Higgs", "l") # line

    # find the y range and set nice axis limits around it
    ymin = max(data_hist.GetMinimum(),0.001)*0.2
    ymax = data_hist.GetMaximum()*1.15
    data_hist.GetYaxis().SetRangeUser(ymin, ymax)
    
    # actually draw the histograms in the canvas
    data_hist.Draw("PE0X0") # plot this first to set the y range
    sig_hist.Draw("HIST same")

    # functional fit for background function.
    # https://root.cern/doc/master/classTH1.html#a63eb028df86bc86c8e20c989eb23fb2a: predefined fucntions: gaus, landau, expo, pol1,...,9
    # TODO try a 4th order polynominal instead of the exponential, or your own user-defined function.
    # define a bg hist, starting from the data
    bg_hist = data_hist.Clone("bg")
    # define the fit object and settings
    bg_fit = TF1("bg_fit", "expo", xrange[0], xrange[1])
    
    # A bit messy in python but defining excluded range if we want to blind the function
    frange = r.Fit.DataRange()
    frange.AddRange(0, xrange[0], range_to_blind[0])
    frange.AddRange(0, range_to_blind[1], xrange[1])
    # Yes this is a function within a function but it's not really a problem
    def blindFit(x, p):
        if not (frange.IsInside(x[0])):
            TF1.RejectPoint()
            return 0;
        return bg_fit.EvalPar(x,p)

    blind_fit = TF1("blind_fit", blindFit, xrange[0], xrange[1], bg_fit.GetNpar())
    blind_fit.SetLineColor(r.kBlue)
    blind_fit.SetLineWidth(3)
    blind_fit.SetMarkerSize(0)

    # indicate the region blinded
    line = r.TLine()
    line.SetLineColor(r.kGray)
    line.SetLineWidth(3)
    line.DrawLine(range_to_blind[0], ymin, range_to_blind[0], ymax/1.15)
    line.DrawLine(range_to_blind[1], ymin, range_to_blind[1], ymax/1.15)
    l=TLatex()
    l.SetNDC()
    l.SetTextFont(42)
    l.SetTextColor(r.kGray)
    l.SetTextSize(0.03)
    left = (range_to_blind[0] - xrange[0])/(xrange[1] - xrange[0])
    l.DrawLatex(left + 0.14, 0.7, "Blind in bg fit")

    bg_fit.SetLineColor(r.kBlue)
    bg_fit.SetLineWidth(3)
    bg_hist.SetMarkerSize(0)
    bg_fit.SetMarkerSize(0)

    # perform the fit and print out the results (note 0 option stops it being drawn automatically in the wrong colour)
    # save the parameters and errors in case we need them later...
    bg_hist.Fit("blind_fit","0") 
    nparams = blind_fit.GetNpar()
    params = blind_fit.GetParameters()
    errors = blind_fit.GetParErrors()
    for p in range(nparams):
        print("parameter {}:".format(p))
        print("value: "+str(params[p]), ", error: "+str(errors[p]))

    bg_hist.Draw("FUNC same")

    # new function to plot extapolate to the blinded region.
    plot_fit = TF1("plot_fit", "expo", xrange[0], xrange[1])
    plot_fit.SetParameters(blind_fit.GetParameters())
    plot_fit.SetLineColor(r.kBlue)
    plot_fit.Draw("same")

    legend.AddEntry(plot_fit, "SM Background fit","l")

    # draw the legend
    legend.Draw("same")

    # add some text
    l=TLatex()
    l.SetNDC()
    l.SetTextFont(42)
    l.SetTextColor(r.kBlack)
    l.SetTextSize(0.036)
    lumi_string = '10 fb^{#minus1} #sqrt{s}=13 TeV'
    l.DrawLatex(0.2, 0.9, lumi_string)
    l.SetTextColor(r.kBlue)
    # TODO make this not wrong/crash if we want to change the fit function:
    l.DrawLatex(0.2, 0.85, "fit: y = exp({0:.2f} + {1:.2f}*x)".format(params[0], params[1]))
 
    pad1.RedrawAxis()

    # now lets move to the ratio pad to draw the data/bg and signal/bg.
    pad2.cd()
    
    data_ratio = data_hist.Clone("data ratio")
    data_ratio.Divide(plot_fit)
    data_ratio.GetYaxis().SetRangeUser(0.01, 1.99)
    data_ratio.GetYaxis().SetTitle("distribution / SM bg")
   
    data_ratio.GetXaxis().SetTitleSize(0.1)
    data_ratio.GetXaxis().SetTitleOffset(1.3)
    data_ratio.GetXaxis().SetLabelSize(0.1)

    data_ratio.GetYaxis().SetTitleSize(0.1)
    data_ratio.GetYaxis().SetTitleOffset(0.45)
    data_ratio.GetYaxis().SetLabelSize(0.1)
    data_ratio.GetYaxis().SetNdivisions(7)



    data_ratio.Draw("PE0X0")
  
    line = r.TLine()
    line.SetLineColor(r.kBlue)
    line.SetLineWidth(3)
    line.DrawLine(xrange[0], 1., xrange[1], 1.)

    sig_ratio = sig_hist.Clone("sig ratio")
    sig_ratio.Divide(plot_fit)
    sig_ratio.Draw("HIST same")

    exts = [".pdf", ".png"]
    for ext in exts: canvas.SaveAs(plotdir + "SigDataBgFit_" + variable + ext)


In [27]:
variable = "diphoton_mass"
# do we want to blind the visual of the plot?
blind = True
# want to blind the signal part of the fit in any case so we don't fit the background to bits of data that might have signal in.
range_to_blind = [105., 145.] #GeV
#this will determine the axis range and the range used in the fit.
fit_range = [90., 300.] #GeV
# TODO can you alter the function to allow us to rebin the histograms?

# read in the signal and data histograms
sig_file = TFile(hist_path + "allHiggs.root", "READ")
data_file = TFile(hist_path + "data.root", "READ")
sig_hist = sig_file.Get(variable)
data_hist = data_file.Get(variable)

plotSigBgData(sig_hist, data_hist, fit_range, plot_path, variable, blind, range_to_blind)

106 146
parameter 0:
value: 13.874847395588159 , error: 0.002915443339861861
parameter 1:
value: -0.02865600068520325 , error: 2.2108317684994145e-05
 FCN=60052 FROM MIGRAD    STATUS=CONVERGED     142 CALLS         143 TOTAL
                     EDM=2.84906e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.38748e+01   2.91544e-03   9.64610e-05  -3.84264e-02
   2  p1          -2.86560e-02   2.21083e-05   7.31477e-07  -1.75613e+00


Warning in <TH1F::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1F::Sumw2>: Sum of squares of weights structure already created
Info in <TCanvas::Print>: pdf file ../plots/GamGam_root/SigDataBgFit_diphoton_mass.pdf has been created
Info in <TCanvas::Print>: png file ../plots/GamGam_root/SigDataBgFit_diphoton_mass.png has been created
